In [20]:
import akshare as ak
import pandas as pd

In [21]:
# 茅台
symbol_id = "600519"
stock_record = ak.stock_zh_a_hist(symbol=symbol_id, adjust="hfq")

stock_record.to_csv("{}.csv".format(symbol_id))

In [22]:
stock_record = pd.read_csv(f"{symbol_id}.csv", index_col='日期', parse_dates=["日期"])[['开盘', '收盘', '最高', '最低']]
# 修改索引和column的名字
stock_record.index.name = 'date'
stock_record.columns = ['open', 'close', 'high', 'low']

In [23]:
# 收盘比开盘上涨3%及以上的日期
stock_record[(stock_record['close'] - stock_record['open'])/stock_record['open'] >= 0.03].index

DatetimeIndex(['2001-08-27', '2001-08-28', '2001-09-10', '2001-12-21',
               '2002-01-18', '2002-01-31', '2003-01-14', '2003-10-29',
               '2004-01-05', '2004-01-14',
               ...
               '2021-09-27', '2021-10-13', '2021-12-08', '2021-12-23',
               '2022-06-17', '2022-08-31', '2022-11-01', '2022-11-04',
               '2022-11-15', '2023-01-05'],
              dtype='datetime64[ns]', name='date', length=320, freq=None)

In [24]:
# 开盘比前日收盘跌2%及以上的日期
# shift：+ 数据下移 - 数据上移
stock_record[(stock_record['open'] - stock_record['close'].shift(1))/stock_record['close'].shift(1) <= -0.02].index

DatetimeIndex(['2001-09-12', '2002-06-26', '2002-12-13', '2004-07-01',
               '2004-10-29', '2006-08-21', '2006-08-23', '2007-01-25',
               '2007-02-01', '2007-02-06', '2007-03-19', '2007-05-21',
               '2007-05-30', '2007-06-05', '2007-09-10', '2008-03-13',
               '2008-03-17', '2008-03-25', '2008-03-27', '2008-04-22',
               '2008-04-23', '2008-04-29', '2008-05-13', '2008-06-10',
               '2008-06-13', '2008-06-24', '2008-06-27', '2008-08-11',
               '2008-08-19', '2008-09-23', '2008-10-10', '2008-10-15',
               '2008-10-16', '2008-10-20', '2008-10-23', '2008-10-27',
               '2008-11-06', '2008-11-20', '2008-11-21', '2008-12-02',
               '2009-02-27', '2009-03-25', '2010-04-26', '2010-04-30',
               '2011-08-05', '2012-03-27', '2012-08-10', '2012-11-22',
               '2012-12-04', '2012-12-24', '2013-01-16', '2013-01-25',
               '2013-09-02', '2014-04-25', '2015-01-19', '2015-07-08',
      

In [25]:
# 从2010年1月1日开始，每月第一个交易日买入1手，每年最后一个交易日卖出所有，到今天的收益
stock_record = stock_record['2010-01': '2023-12']
stock_record

,open,close,high,low
date,,,,
2010-01-04,664.26,656.49,664.26,654.11
2010-01-05,660.45,654.60,662.38,652.94
2010-01-06,652.90,644.48,654.83,642.78
2010-01-07,644.48,633.01,646.11,626.06
2010-01-08,634.06,626.51,634.06,619.34
...,...,...,...,...
2022-12-30,9427.45,9382.23,9512.83,9382.23
2023-01-03,9403.34,9397.36,9439.66,9276.76
2023-01-04,9397.31,9372.23,9441.02,9326.96


In [26]:
# 每个月的和
stock_record.resample('M').sum()
# 每个月的第一天的数据 M可以写成MS。虽然结果的index是每个月的最后一天，实际取的第一天数据
# last()即最后一天的数据
stock_monthly = stock_record.resample('M').first()
# 每年的最后一天的数据，注意最后一行数据的合理性：最后一天到了没
stock_yearly = stock_record.resample('A').last()[:-1]

In [27]:
cost = 0
# range, 不包括end
for year in range(2010, 2024):
  # 花费，支出
  cost += -(stock_monthly.loc[str(year)]['open'].sum() * 100)
  hold = len(stock_monthly.loc[str(year)]) * 100
  # 今年的数值是没有的！！如果不排除今年的，会报错
  if year != 2023:
    sell = stock_yearly.loc[str(year)]['open'][0] * hold
    cost += sell
cost

5724810.000000001